In [ ]:
model.eval()

#beta = (1 / torch.sqrt(torch.tensor(num_embeddings, dtype=torch.float32)) * (1- (num_training_updates-5)/num_training_updates))

beta = 64
lambd = 0.001
(valid_originals, _) = next(iter(validation_loader))
valid_originals = valid_originals.to(device)

vq_output_eval = model._pre_vq_conv(model._encoder(valid_originals))
_, valid_quantize, _, _,_ = model._vq_vae(vq_output_eval,beta,lambd)
valid_reconstructions = model._decoder(valid_quantize)


In [ ]:
(train_originals, _) = next(iter(training_loader))
train_originals = train_originals.to(device)
_, train_reconstructions, _, _,_ = model._vq_vae(train_originals,beta,lambd)

In [ ]:

def show(img):
    npimg = img.numpy()
    fig = plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    

In [ ]:
show(make_grid(valid_reconstructions.cpu().data)+0.5, )


In [ ]:
show(make_grid(valid_originals.cpu()+0.5))

In [ ]:
proj = umap.UMAP(n_neighbors=3,
                 min_dist=0.1,
                 metric='cosine').fit_transform(model._vq_vae._embedding.weight.data.cpu())


plt.scatter(proj[:,0], proj[:,1], alpha=0.3,label="k-means")
plt.legend()
plt.xlabel(r"$d_1$")
plt.ylabel(r"$d_2$")